```@meta
CurrentModule = AlgorithmicRecourse 
```

# From cat to dog

In [11]:
using Statistics, AlgorithmicRecourse
using Flux, Flux.Optimise
using MLDatasets: CIFAR10
using Images.ImageCore
using Flux: onehotbatch, onecold
using Base.Iterators: partition

In [14]:
include("CIFAR10.jl")

In [ ]:
using Plots
image(x) = colorview(RGB, permutedims(x, (3, 2, 1)))
plot(image(train_x[:,:,:,rand(1:end)]))